In [2]:
import pandas as pd
import numpy as np

In [3]:
# Perfrom Extracting
df = pd.read_csv("../jobs.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   country           399 non-null    object
 1   date_time         399 non-null    object
 2   employment_type   399 non-null    object
 3   num_applications  399 non-null    int64 
 4   num_views         399 non-null    int64 
 5   tools             399 non-null    object
 6   year_of_exp       399 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 21.9+ KB


In [18]:
# Perfrom Cleaning, Wrangling

# to safely evaluate a string containing a Python literal 
from ast import literal_eval

# change year of exp to data type int8
df["year_of_exp"] = df["year_of_exp"].astype(np.int8)

# to datetime objects
df["date_time"] = pd.to_datetime(df["date_time"], format="%H:%M %d.%m.%Y")

# remove rows where there is no tools
df = df[df["tools"] != "[]"]

# convert string list tools column to list
df.tools = df.tools.replace("\\", "/")
# df.tools = df.tools.apply(literal_eval)

# df["tools"].agg(sum)
# df_exploded = df.explode("tools")

# Get unique tools from the exploded DataFrame
# unique_tools = df_exploded["tools"].unique()

df.head(13).tools

0     [Azure Cloud Platform, API, ChatGPT, Git, NLP,...
2     [Docker, React, Middle Front, Technology, Rabb...
3     [React, SSR, DRY, CSS, Jest, Python APIs, Full...
4          [Git, US Timezone, Python, English, CST/EST]
5     [DevOps, Firestore, Google AppEngine, API, Rea...
6                                     [IaC, Kubernetes]
7     [MongoDB, Docker, GitHub CI, Full-Stack, Elast...
8     [SQL, Data Analysis, Excel, Python, ETL, Power...
9     [Tracking Data, Python, Vue, English, State Ma...
10    [PostgreSQL, Data Management, SQL, Project, Re...
11    [SQL, AWS, Python, Software Design, English, G...
12    [PostgreSQL, Nginx, SQL, Docker, Git, HTML, CS...
13    [Clickhouse, Gitlab CI\CD, Server, Dbt, Scient...
Name: tools, dtype: object

In [ ]:
s